In [84]:

from common.read_spectral_common import (
    data_df_13_09_2022, 
    calculate_mutual_info_for_all, 
    generate_sample, create_X_train_Y_train, 
    mutual_info_regression, 
    get_max_bands, 
    get_bands_ix_from_mutual_info, 
    get_average_bands, 
    get_max_bands, 
    get_min_bands,
    predict_using_neutral_network, 
    predict_using_random_forest, 
    predict_using_decision_tree,
    get_full_path,
    load_sklearn_model_to_file_by_cloudpickle,
)
import matplotlib.pyplot as plt
import pickle
import cloudpickle
import common
from sklearn import tree
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import datetime
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

In [85]:

target_value = "K"
train_field = "J"
function_get = get_max_bands

In [86]:
calculate_mutual_info_for_all(data_df_13_09_2022, target_value, train_field, function_get)

[(1, 0.24651027303286766),
 (13, 0.21845936791881027),
 (55, 0.21177624907364834),
 (56, 0.186240614371878),
 (21, 0.1533841487652312),
 (99, 0.15201779721687592),
 (104, 0.15046498601681568),
 (93, 0.14217077591500704),
 (50, 0.14091867735390462),
 (105, 0.1310928005778611),
 (16, 0.12407044818194146),
 (57, 0.11816304739558747),
 (65, 0.11737043021490301),
 (4, 0.11376824242761963),
 (109, 0.11285797629376315),
 (3, 0.10722048381778926),
 (48, 0.10314675416854069),
 (102, 0.10140777393877087),
 (49, 0.10110722803911143),
 (62, 0.09082995057605725),
 (43, 0.08698831122798634),
 (106, 0.08019899337636183),
 (24, 0.0794861046717239),
 (7, 0.07867188467607988),
 (53, 0.06845287191584015),
 (97, 0.06839688844626712),
 (61, 0.06693656247443514),
 (34, 0.06659534856994842),
 (80, 0.06588255455938352),
 (94, 0.06572603811057487),
 (110, 0.06444676041525677),
 (9, 0.06341230597330405),
 (108, 0.06209019864140197),
 (92, 0.05955676733503035),
 (12, 0.05785941866040156),
 (37, 0.056204134874615

In [87]:
bands_ix = get_bands_ix_from_mutual_info(data_df_13_09_2022, 0, target_value, train_field, function_get)

In [88]:
len(bands_ix)

69

In [89]:
# bands_ix = filter(lambda data: data < 100, bands_ix)

In [90]:
bands_ix = list(bands_ix)

In [91]:
bands_ix

[1,
 3,
 4,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 20,
 21,
 24,
 27,
 29,
 34,
 35,
 37,
 38,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 53,
 55,
 56,
 57,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 69,
 71,
 72,
 73,
 76,
 77,
 80,
 83,
 87,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 101,
 102,
 104,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 114,
 117,
 121]

In [92]:
len(bands_ix)

69

In [ ]:
date_now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
NN_file_path = get_full_path(f"../../model_saved/NN_save/NN_object/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
RF_file_path = get_full_path(f"../../model_saved/RF_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
DT_file_path = get_full_path(f"../../model_saved/DT_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")

In [ ]:
sample = generate_sample(data_df_13_09_2022, bands_ix, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample, bands_ix)
sample_target = generate_sample(data_df_13_09_2022, bands_ix, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target, bands_ix)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 130}
re_run = "Y"
loss_NN, pred_NN, NN_model = predict_using_neutral_network(
    X_train, 
    Y_train, 
    X_target, 
    Y_target, 
    bands_ix,
    NN_file_path, 
    super_param, 
    re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, RF_model = predict_using_random_forest(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, DT_model = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

In [93]:
cloudpickle.register_pickle_by_value(common)

with open(NN_file_path, "wb") as file:
    cloudpickle.dump(NN_model, file)
    print(f"Save NN object to: {NN_file_path}")
with open(RF_file_path, "wb") as file:
    cloudpickle.dump(RF_model, file)
    print(f"Save RF object to: {RF_file_path}")
with open(DT_file_path, "wb") as file:
    cloudpickle.dump(DT_model, file)
    print(f"Save DT object to: {DT_file_path}")

Save NN object to: D:\code\model_saved\NN_save\NN_object\J_predict_K_2024-03-05-09-13-30_using_get_max_bands.pkl
Save RF object to: D:\code\model_saved\RF_save\J_predict_K_2024-03-05-09-13-30_using_get_max_bands.pkl
Save DT object to: D:\code\model_saved\DT_save\J_predict_K_2024-03-05-09-13-30_using_get_max_bands.pkl


In [94]:
avarage_target_value = np.average(Y_target)
loss_NN = float(loss_NN)
df_report = pd.DataFrame(
    {
        "date_time": [datetime.now()],
        "train_field": [str(train_field)],
        "target_value": [str(target_value)],
        "bands_ix": [str(bands_ix)],
        "total_band": [len(bands_ix)],
        "function_get": [function_get.__name__],
        "loss_NN": [loss_NN],
        "% loss_NN": [loss_NN / avarage_target_value * 100],
        "loss_RF": [loss_RF],
        "% loss_RF": [loss_RF / avarage_target_value * 100],
        "loss_DT": [loss_DT],
        "% loss_DT": [loss_DT / avarage_target_value * 100],
        "average_target_value": [avarage_target_value],
        "File save NN": [NN_file_path],
        "File save RF": [RF_file_path],
        "File save DT": [DT_file_path],
    }
)

In [95]:
df_report

,date_time,train_field,target_value,bands_ix,total_band,function_get,loss_NN,% loss_NN,loss_RF,% loss_RF,loss_DT,% loss_DT,average_target_value,File save NN,File save RF,File save DT
0,2024-03-05 09:13:30.371898,J,K,"[1, 3, 4, 7, 9, 10, 11, 12, 13, 14, 15, 16, 20...",69,get_max_bands,3330.085693,26.403851,4494.703527,35.637967,5116.632876,40.56917,12612.121094,D:\code\model_saved\NN_save\NN_object\J_predic...,D:\code\model_saved\RF_save\J_predict_K_2024-0...,D:\code\model_saved\DT_save\J_predict_K_2024-0...


In [96]:
def append_df_to_excel(df: pd.DataFrame, file_path, sheet_name="Sheet1"):
    if not os.path.exists(file_path):
        df.to_excel(
            file_path,
            sheet_name=sheet_name,
            index=False,
            header=True
        )
        return
    
    wb = load_workbook(file_path)
    ws = wb[sheet_name]
    for r in dataframe_to_rows(df, index=False, header=False):
        ws.append(r)
    wb.save(file_path)

In [97]:
try:
    report_file_path = get_full_path("../../report/agriculture_report.xlsx")
    sheet_name = "Sheet1"

    append_df_to_excel(df_report, report_file_path, sheet_name)
    print(f"SUCCESS: DATA IS WRITED TO FILE {report_file_path}")
except PermissionError as err:
    print(f"ERROR: YOU ARE OPENNING FILE {report_file_path}")

SUCCESS: DATA IS WRITED TO FILE D:\code\report\agriculture_report.xlsx
